In [ ]:
import os
import keras
from keras.applications import inception_v3 as inc_net
from keras.preprocessing import image
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from skimage.segmentation import mark_boundaries
from skimage.segmentation import find_boundaries
%load_ext autoreload
%autoreload 2
import sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image
import time

In [ ]:
column_to_label = {1: 'airplane', 2: 'bird', 3: 'car', 4: 'cat', 5: 'deer', 
                   6: 'dog', 7: 'horse', 8: 'monkey', 9: 'ship', 10: 'truck'}

# Using Inception
Here we create a standard InceptionV3 pretrained model and use it on images by first preprocessing them with the preprocessing tools

In [ ]:
inet_model = inc_net.InceptionV3(weights=None, include_top=False)

In [ ]:
import cv2
import glob

data = []
labels = []
for i in range(1, 11):
    img_dir = "img/" + str(i)
    data_path = os.path.join(img_dir,'*g')
    files = glob.glob(data_path)
    for f1 in files:
        img = cv2.imread(f1)
        info = np.iinfo(img.dtype) # Get the information of the incoming image type
        img = img.astype(np.float64) / info.max # normalize the data to 0 - 1
        img = img * 2 - 1
        data.append(img)
        labels.append(i)

rand_order = np.arange(3180)
np.random.shuffle(rand_order)

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
# References: 
# https://github.com/keras-team/keras/issues/9214
# https://keras.io/applications/#usage-examples-for-image-classification-models

from keras.applications import resnet50, inception_v3, vgg16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.optimizers import Adam

num_samples = 3180
batch_size = 10
num_classes = 10
num_batches = num_samples // batch_size
iteration = 0

base_model = inet_model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.0001), 
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
class SubPlotter:
        
    def __init__(self, num_batches):
        self.i = 0
        self.start = time.time()
        self.fig, self.axeslist = plt.subplots(ncols=6, nrows=num_batches, figsize=(96, 96 * num_batches))
        
    def plot_row(self, imgs, _labels):
        for i in range(len(imgs)):
            self._plot_two(imgs[i], _labels[i])

    def _plot_two(self, img, label):
        # Create Explanation
        explainer = lime_image.LimeImageExplainer()
        # what is this /2+.5 nonsense?!?
        expl = explainer.explain_instance(img, model.predict, top_labels=1, hide_color=0, num_samples=1000)
        
        # Display superpixel mapping
        outline = find_boundaries(expl.segments)
        c = img.copy()
        c[outline] = (1, 1, 0)
        _sp = self.axeslist.ravel()[self.i]
        _sp.imshow(c / 2 + 0.5, cmap=plt.gray())
        _sp.set_axis_off()
        self.i += 1
        
        # Display superpixel contrib
        temp, mask = expl.get_image_and_mask(expl.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
        _sp = self.axeslist.ravel()[self.i]
        _sp.imshow(mark_boundaries(temp / 2 + 0.5, mask), cmap=plt.gray())
        
        preds = model.predict(img[np.newaxis,])
        label_id = np.argmax(preds) + 1
        label_name = column_to_label[label_id]
    
        _sp.set_axis_off()
        self.i += 1
        
        print("Is {}, Predicted {}".format(column_to_label[label], label_name))
        
    def print_preds(self, alter_image):
        preds = inet_model.predict(np.array([alter_image]))
        tup = decode_predictions(preds)[0][0]
        print("\noriginal model behavior")
        for x in decode_predictions(preds)[0]:
            print(x[1], x[2])
            
    def add_img(self, img, title):
        _sp = self.axeslist.ravel()[self.i]
        _sp.imshow(img, cmap=plt.gray())
        _sp.set_title("num_samples = " + str(title))
        _sp.set_axis_off()
        self.i += 1
        
    def end(self):
        plt.tight_layout()
        plt.show()
        end = time.time()
        diff = end - self.start
        print("Ran for {} seconds!! ({} minutes)".format(round(diff, 3), round(diff / 60, 3)))

In [ ]:
selection = rand_order[-180:]
val_images = np.array([data[index] / 2 + 0.5 for index in selection])
val_labels = np.array([labels[index] / 2 + 0.5 for index in selection])

In [ ]:
# Change input to 53
sp = SubPlotter(1)

for iteration in range(1):
    start = iteration * 10
    end = (iteration + 1) * 10
    selection = rand_order[start: end]
    
    # / 2 + 0.5 gets rid of the nan?!?
    x_train_batch = np.array([data[index] / 2 + 0.5 for index in selection])
    y_train_batch = np.array([labels[index] / 2 + 0.5 for index in selection])
    
    print(str(model.evaluate(x_train_batch, y_train_batch, batch_size=batch_size, verbose=0)) + "\n")
    model.fit(x_train_batch, y_train_batch,
              epochs=1,
              batch_size=batch_size,
              shuffle=False,
              validation_data=(val_images, val_labels))
    
    _start = time.time()
    sp.plot_row(data[:3], labels[:3])
    _end = time.time()
    _diff = _end - _start
    print("Ran for {} seconds!! ({} minutes)".format(round(_diff, 3), round(_diff / 60, 3)))
    
sp.end()

In [ ]:
sp = SubPlotter(53)

for iteration in range(53, 106):
    start = iteration * 10
    end = (iteration + 1) * 10
    selection = rand_order[start: end]
    x_train_batch = np.array([data[index] for index in selection])
    y_train_batch = np.array([labels[index] for index in selection])

    print(str(model.evaluate(x_train_batch, y_train_batch, batch_size=batch_size, verbose=0)) + "\n")
    model.fit(x_train_batch, y_train_batch,
              epochs=1,
              batch_size=batch_size,
              shuffle=False,
              validation_data=(val_images, val_labels))
    
    _start = time.time()
    sp.plot_row(data[:3], labels[:3])
    _end = time.time()
    _diff = _end - _start
    print("Ran for {} seconds!! ({} minutes)".format(round(_diff, 3), round(_diff / 60, 3)))
    
sp.end()

In [ ]:
sp = SubPlotter(53)

for iteration in range(106, 159):
    start = iteration * 10
    end = (iteration + 1) * 10
    selection = rand_order[start: end]
    x_train_batch = np.array([data[index] for index in selection])
    y_train_batch = np.array([labels[index] for index in selection])

    print(str(model.evaluate(x_train_batch, y_train_batch, batch_size=batch_size, verbose=0)) + "\n")
    model.fit(x_train_batch, y_train_batch,
              epochs=1,
              batch_size=batch_size,
              shuffle=False,
              validation_data=(val_images, val_labels))
    
    _start = time.time()
    sp.plot_row(data[:3], labels[:3])
    _end = time.time()
    _diff = _end - _start
    print("Ran for {} seconds!! ({} minutes)".format(round(_diff, 3), round(_diff / 60, 3)))
    
sp.end()

In [ ]:
sp = SubPlotter(53)

for iteration in range(159, 212):
    start = iteration * 10
    end = (iteration + 1) * 10
    selection = rand_order[start: end]
    x_train_batch = np.array([data[index] for index in selection])
    y_train_batch = np.array([labels[index] for index in selection])

    print(str(model.evaluate(x_train_batch, y_train_batch, batch_size=batch_size, verbose=0)) + "\n")
    model.fit(x_train_batch, y_train_batch,
              epochs=1,
              batch_size=batch_size,
              shuffle=False,
              validation_data=(val_images, val_labels))
    
    _start = time.time()
    sp.plot_row(data[:3], labels[:3])
    _end = time.time()
    _diff = _end - _start
    print("Ran for {} seconds!! ({} minutes)".format(round(_diff, 3), round(_diff / 60, 3)))
    
sp.end()

In [ ]:
sp = SubPlotter(53)

for iteration in range(212, 265):
    start = iteration * 10
    end = (iteration + 1) * 10
    selection = rand_order[start: end]
    x_train_batch = np.array([data[index] for index in selection])
    y_train_batch = np.array([labels[index] for index in selection])

    print(str(model.evaluate(x_train_batch, y_train_batch, batch_size=batch_size, verbose=0)) + "\n")
    model.fit(x_train_batch, y_train_batch,
              epochs=1,
              batch_size=batch_size,
              shuffle=False,
              validation_data=(val_images, val_labels))
    
    _start = time.time()
    sp.plot_row(data[:3], labels[:3])
    _end = time.time()
    _diff = _end - _start
    print("Ran for {} seconds!! ({} minutes)".format(round(_diff, 3), round(_diff / 60, 3)))
    
sp.end()

In [ ]:
sp = SubPlotter(53)

for iteration in range(265, 318):
    start = iteration * 10
    end = (iteration + 1) * 10
    selection = rand_order[start: end]
    x_train_batch = np.array([data[index] for index in selection])
    y_train_batch = np.array([labels[index] for index in selection])

    print(str(model.evaluate(x_train_batch, y_train_batch, batch_size=batch_size, verbose=0)) + "\n")
    model.fit(x_train_batch, y_train_batch,
              epochs=1,
              batch_size=batch_size,
              shuffle=False,
              validation_data=(val_images, val_labels))
    
    _start = time.time()
    sp.plot_row(data[:3], labels[:3])
    _end = time.time()
    _diff = _end - _start
    print("Ran for {} seconds!! ({} minutes)".format(round(_diff, 3), round(_diff / 60, 3)))
    
sp.end()

## Trash Dump